In [33]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

Hyperparameters

In [34]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

Full Implementation

In [35]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [36]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295765  [    0/60000]
loss: 2.287457  [ 6400/60000]
loss: 2.269001  [12800/60000]
loss: 2.270285  [19200/60000]
loss: 2.258187  [25600/60000]
loss: 2.221064  [32000/60000]
loss: 2.233886  [38400/60000]
loss: 2.195761  [44800/60000]
loss: 2.186494  [51200/60000]
loss: 2.168061  [57600/60000]
Test Error: 
 Accuracy: 49.1%, Avg loss: 2.162925 

Epoch 2
-------------------------------
loss: 2.167405  [    0/60000]
loss: 2.158584  [ 6400/60000]
loss: 2.108784  [12800/60000]
loss: 2.126428  [19200/60000]
loss: 2.084460  [25600/60000]
loss: 2.023107  [32000/60000]
loss: 2.045988  [38400/60000]
loss: 1.970315  [44800/60000]
loss: 1.966621  [51200/60000]
loss: 1.911201  [57600/60000]
Test Error: 
 Accuracy: 60.2%, Avg loss: 1.904969 

Epoch 3
-------------------------------
loss: 1.930552  [    0/60000]
loss: 1.899353  [ 6400/60000]
loss: 1.794306  [12800/60000]
loss: 1.836722  [19200/60000]
loss: 1.733983  [25600/60000]
loss: 1.685872  [32000/600